# Initialize OpenAI Client

In [ ]:
client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")

# Split video to frames

In [ ]:
# Load the cricket video
video = cv2.VideoCapture("multievent2.mov")

# Calculate video length
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = video.get(cv2.CAP_PROP_FPS)
video_length_seconds = length / fps

print(f'Video length: {video_length_seconds:.2f} seconds')

# Read frames and encode to base64
base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# Release the video object
video.release()
print(len(base64Frames), "frames read.")


# Multiple Event commentary generation prompt engineering test with every 30th frame being passed

* Here we pass frames belonging to not just one,but multiple events.
* We need more number of frames to be processed by a larger, stronger model (GPT-4o) to be able to do the processing.

In [ ]:
# Create OpenAI chat completion for cricket commentary
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user", 
            "content": [
                f"This video contains multiple cricket deliveries, each lasting a few seconds. Based on the sequence of frames provided, generate a separate short cricket commentator-style voiceover line for each delivery. Number them 1., 2., 3., etc. Focus only on the delivery, the shot, and the outcome. Do not mention the player names or identify them or describe visuals like the crowd.",
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::30]),
            ]
        }
    ],
    max_tokens=100,
)

# Print the generated cricket commentary
cricket_commentary = response.choices[0].message.content
print(cricket_commentary)
